# 토스 스타일 모바일 UI — 단계별 실습 노트

이 노트북은 **제공해 주신 HTML/CSS 전체 코드**를 1단계씩 조립하며 완성해 보는 수업 자료입니다.  
각 단계 셀을 순서대로 실행하면, 같은 폴더에 `index_stepN.html` 파일이 생성되고 미리보기를 즉시 확인할 수 있습니다.  
마지막에는 **원문 그대로의 전체 HTML/CSS**를 `index.html`로 작성/검증합니다.

> **실행 순서**  
> 1) 0단계 준비 셀 실행 → 2) 단계별 셀을 위에서 아래로 실행 → 3) 마지막 ‘최종 조립’ 셀 실행

---

## 학습 목표
- 디자인 토큰(변수)와 공통 카드 레이아웃 이해
- **컨테이너 쿼리 + `cqi` 단위**와 `clamp()`를 이용한 반응형 타이포그래피
- 리스트/배너/네비게이션 등 **재사용 가능한 컴포넌트 설계**
- 모든 브라우저를 위한 **Fallback → Enhanced 전략** 적용

## 폴더 구조(자동 생성)
```
toss_ui_lesson/
 ├─ index.html            # 최종 완성본 (원문 그대로)
 ├─ index_step1.html      # 단계별 산출물
 ├─ index_step2.html
 ├─ ...
 └─ toss_main_CSS.CSS     # 제공된 전체 CSS
```


In [ ]:
# 0) 준비: 실습 파일 경로/유틸 정의 및 CSS 쓰기
import os, io, sys, json, textwrap
from IPython.display import HTML, display

BASE = "toss_ui_lesson"
CSS_PATH = f"{BASE}/toss_main_CSS.CSS"

os.makedirs(BASE, exist_ok=True)

css_full = r"""/* =========================
   STEP 1: CSS 변수/베이스 세팅
   - 테마 색상 변수, 그림자, radius
   - 박스사이징, 폰트, 배경
========================== */
:root{
  --bg: #f3f5f7;
  --card: #ffffff;
  --text: #121418;
  --muted: #6b7785;
  --blue: #2f6df6;
  --blue-ink:#2a5fe0;
  --line: #e7ebf0;
  --good: #10a37f;
  --warn: #ef4444;
  --gold: #ffbf37;
  --shadow: 0 8px 24px rgba(16,24,40,.06);
  --radius: 20px;
}
*{box-sizing:border-box}
html,body{
  height:100%;
  background:var(--bg);
  color:var(--text);
  font-family:Inter,system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Apple SD Gothic Neo","Noto Sans KR",sans-serif
}

/* =========================
   STEP 2: 앱 컨테이너
   - 화면 중앙에 카드형 래퍼
========================== */
.app{
  max-width: 420px; /* iPhone-ish width */
  margin: 24px auto 48px;
  padding: 16px 16px 100px;
  background: linear-gradient(#f7f9fb, #eef2f6);
  border-radius: 32px;
  box-shadow: 0 10px 40px rgba(16,24,40,.08);
  border: 1px solid #eef1f5;
}

/* =========================
   STEP 3: 상태바/헤더
   - Flex 정렬, 작은 배지(pill)
========================== */
.statusbar{
  display:flex;justify-content:space-between;align-items:center;
  font-weight:600; font-size:14px; color:#0b1220; margin-bottom:8px;
}
.statusbar .icons{display:flex;gap:10px;align-items:center;color:#0b1220}
.pill{
  display:inline-flex;align-items:center;gap:8px;background:#e9eef7;border-radius:999px;color:#223; padding:6px 10px;font-weight:600;font-size:13px
}
.header-row{display:flex;justify-content:space-between;align-items:center;margin:8px 0 12px}
.right-btn{display:inline-flex;gap:8px;align-items:center}

/* =========================
   STEP 4: 공통 카드 스타일
========================== */
.card{background:var(--card);border-radius:var(--radius);box-shadow:var(--shadow);padding:18px}
.card+.card{margin-top:12px}

/* =========================
   STEP 5: 컨테이너 쿼리 준비
   - 폭 줄 때 내부 폰트 자동 축소(cqi) 사용을 위해
   - 카드/행을 컨테이너로 지정
========================== */
.card,.small-card,.list .item,.header-row,.row{container-type:inline-size}

/* =========================
   STEP 6: 계좌 상단 카드
   - 제목/잔액/버튼
   - clamp + cqi로 반응형 글자 크기
========================== */
.account-main{display:flex;justify-content:space-between;align-items:center}
.account-title{
  display:flex;align-items:center;gap:10px;color:#51607a;font-weight:700;
  white-space:nowrap;
  font-size:clamp(12px,7cqi,14px);
}
.logo{width:28px;height:28px;background:var(--blue);border-radius:8px;box-shadow:0 4px 10px rgba(47,109,246,.35)}
.balance{
  font-weight:800;letter-spacing:-.3px;margin:6px 0 2px;
  white-space:nowrap;
  font-size:clamp(20px,12cqi,28px);
}
.sub{color:#6b7785;font-weight:600}
.btn{background:#eef3ff;color:var(--blue-ink);border:none;border-radius:14px;padding:8px 14px;font-weight:800;cursor:pointer}
.btn:active{transform:translateY(1px)}

/* =========================
   STEP 7: 카드 내 행(Row)
   - 좌/우 키-값 정렬
   - 우측 내용 길어질 때 elipsis 안전망
========================== */
.row{
  display:flex;justify-content:space-between;align-items:center;padding:12px 0;border-top:1px dashed var(--line);
}
.row:first-child{border-top:none}
.row .k{color:#44536a;font-weight:700;white-space:nowrap}
.row .v{
  color:#66758a;font-weight:700;white-space:nowrap;min-width:0;
  font-size:clamp(12px,8cqi,14px);
  overflow:hidden;text-overflow:ellipsis;
}

/* =========================
   STEP 8: 자주 쓰는 계좌 리스트
   - 리스트 아이템, 칩, 탭
   - 금액/문구 자동 축소
========================== */
.section-title{
  font-weight:800;margin:14px 0 10px;
  white-space:nowrap;
  font-size:clamp(14px,8cqi,18px);
}
.list .item{display:flex;align-items:center;justify-content:space-between;padding:12px 6px;border-bottom:1px solid var(--line)}
.list .item:last-child{border-bottom:none}
.chip{width:34px;height:34px;border-radius:10px;background:#fff6e1;display:grid;place-items:center;box-shadow:inset 0 0 0 1px #ffe6ad}
.chip.pink{background:#ffecef;box-shadow:inset 0 0 0 1px #ffd1db}
.chip.gold{background:#fff6e1}
.name{
  display:flex;align-items:center;gap:12px;font-weight:800;
  min-width:0;white-space:nowrap;
  font-size:clamp(13px,7.5cqi,16px);
}
.right{display:flex;align-items:baseline;gap:6px}
.amt{
  font-weight:900;white-space:nowrap;
  font-size:clamp(16px,9cqi,22px);
}
.pct{font-weight:900;color:var(--warn)}
.muted{color:var(--muted);font-weight:700}
.row-tabs{display:grid;grid-template-columns:repeat(4,1fr);gap:8px;margin-top:10px}
.tab{
  background:#f1f4f9;border:1px solid #e7ebf0;border-radius:14px;padding:10px 12px;text-align:center;font-weight:800;color:#42526e;
  white-space:nowrap;font-size:clamp(12px,7cqi,14px);
}

/* =========================
   STEP 9: 지출/카드값 카드(스몰카드)
   - 좌/우 정렬, 강조 수치
========================== */
.small-card{
  display:flex;justify-content:space-between;align-items:center;background:var(--card);padding:16px;border-radius:var(--radius);box-shadow:var(--shadow)
}
.small-card .title{color:#617089;font-weight:700;white-space:nowrap;font-size:clamp(12px,7cqi,14px)}
.small-card .big{
  font-weight:900;margin-bottom:4px;white-space:nowrap;
  font-size:clamp(18px,10cqi,24px);
}
.badge{background:#eef3ff;color:var(--blue-ink);font-weight:800;border-radius:12px;padding:8px 12px;white-space:nowrap}

/* =========================
   STEP 10: 하단 네비게이션
   - sticky nav
========================== */
.navbar{position:sticky;bottom:0;left:0;right:0;margin:16px -16px -16px;-webkit-backdrop-filter:blur(10px);backdrop-filter:blur(10px);background:rgba(255,255,255,.85);border-top:1px solid var(--line);display:flex;justify-content:space-around;padding:12px 4px}
.navitem{display:flex;flex-direction:column;align-items:center;gap:6px;color:#5b6a7f;font-weight:700;font-size:12px;white-space:nowrap}
.dot{width:22px;height:22px;border-radius:6px;background:#e8eefc}
.navitem.active{color:var(--blue-ink)}

/* =========================
   STEP 11: 혜택 섹션(배너/그리드/리스트)
========================== */
.screen{margin-top:16px}
.divider{height:12px}
.btn-row{display:grid;grid-template-columns:repeat(3,1fr);gap:8px;margin-top:12px}
.btn-lite{background:#f5f7fb;border:1px solid #e6ebf5;border-radius:14px;padding:12px 10px;text-align:center;font-weight:800;color:#40526e;white-space:nowrap}
.banner{display:flex;align-items:center;gap:14px;background:#fff;border-radius:16px;padding:16px;box-shadow:var(--shadow);border:1px solid #eef1f5}
.banner .mark{width:40px;height:40px;border-radius:12px;background:#e8ecff;display:grid;place-items:center;font-weight:900;color:#2a47c9}
.headline{font-weight:900;white-space:nowrap;font-size:clamp(14px,8cqi,16px)}
.row-head{font-weight:900;margin:16px 0 8px;white-space:nowrap;font-size:clamp(14px,8cqi,16px)}
.grid-two{display:grid;grid-template-columns:1fr 1fr;gap:10px}
.tile{background:#fff;border-radius:14px;padding:16px;box-shadow:var(--shadow);border:1px solid #edf0f6;min-height:92px;display:flex;align-items:flex-end;white-space:nowrap;font-size:clamp(13px,7.5cqi,15px)}
.list-simple .li{display:flex;align-items:center;gap:12px;padding:14px 2px;border-bottom:1px solid var(--line);white-space:nowrap;font-size:clamp(13px,7.5cqi,15px)}
.list-simple .li:last-child{border-bottom:none}
.icon-square{width:26px;height:26px;border-radius:8px;background:#e9eef9}

/* =========================
   STEP 12: 여백 유틸리티
========================== */
.sp-8{height:8px}
.sp-16{height:16px}
.sp-24{height:24px}

/* =========================
   STEP 13: 매우 좁은 기기 보정 (선택)
   - 극단적 폭에서 폰트 하한 보정
========================== */
@media (max-width:360px){
  .amt{font-size:20px}
  .balance{font-size:26px}
}


/* ───────────────────────────────────────────────────────────
 1) Fallback (모든 브라우저에서 적용)
 - 컨테이너 쿼리를 지원하지 않는 환경을 위해
 - vw, @media 를 이용해 "대략적인" 수축/확장을 구현
 - 컨테이너 크기 기준은 아니지만, 최소/최대는 보장
 ─────────────────────────────────────────────────────────── */
.account-title{ white-space:nowrap; font-size:clamp(12px, 2.8vw, 14px); }
.balance{       white-space:nowrap; font-size:clamp(20px, 5.8vw, 28px); }
.small-card .big{                font-size:clamp(18px, 5.0vw, 24px); }
.section-title{                   font-size:clamp(14px, 3.6vw, 18px); }
.name{                            font-size:clamp(13px, 3.5vw, 16px); }
.amt{                             font-size:clamp(16px, 4.5vw, 22px); }
.row .v{                          font-size:clamp(12px, 3.0vw, 14px); }
.tab{                             font-size:clamp(12px, 3.0vw, 14px); }
.small-card .title{               font-size:clamp(12px, 3.0vw, 14px); }
.headline,.row-head{              font-size:clamp(14px, 3.4vw, 16px); }
.tile,.list-simple .li{           font-size:clamp(13px, 3.2vw, 15px); }

/* 아주 좁은 기기에서(예: 360px 이하) 하한선 살짝 낮춤 */
@media (max-width: 360px){
  .amt{ font-size:clamp(14px, 5.2vw, 22px); }
  .balance{ font-size:clamp(18px, 6.6vw, 28px); }
}

/* ───────────────────────────────────────────────────────────
 2) Enhanced (컨테이너 쿼리를 지원하는 브라우저에서만 덮어쓰기)
 - container-type을 선언하고 cqi 단위로 더 정밀하게 반응
 - 동일한 선택자에 대해 아래 규칙이 fallback을 override
 ─────────────────────────────────────────────────────────── */
@supports (container-type: inline-size) {
  /* 컨테이너 선언: 이들 내부에서 cqi 사용 가능 */
  .card, .small-card, .list .item, .header-row, .row {
    container-type: inline-size;
  }

  .account-title{ font-size:clamp(12px, 7cqi, 14px); }
  .balance{       font-size:clamp(20px, 12cqi, 28px); }
  .small-card .big{                font-size:clamp(18px, 10cqi, 24px); }
  .section-title{                   font-size:clamp(14px, 8cqi, 18px); }
  .name{                            font-size:clamp(13px, 7.5cqi, 16px); }
  .amt{                             font-size:clamp(16px, 9cqi, 22px); }
  .row .v{                          font-size:clamp(12px, 8cqi, 14px); }
  .tab{                             font-size:clamp(12px, 7cqi, 14px); }
  .small-card .title{               font-size:clamp(12px, 7cqi, 14px); }
  .headline, .row-head{             font-size:clamp(14px, 8cqi, 16px); }
  .tile, .list-simple .li{          font-size:clamp(13px, 7.5cqi, 15px); }
}"""  # placeholder to be replaced dynamically

In [ ]:
# CSS 파일 저장 (항상 동일 파일을 사용)
with open(CSS_PATH, "w", encoding="utf-8") as f:
    f.write(css_full)

print("✔ CSS 저장 완료 →", CSS_PATH)

In [ ]:
# 미리보기 유틸: <link> 대신 <style>로 CSS를 인라인하여 노트북 내에서도 동일 렌더링 보장
def inline_preview(html_str: str) -> HTML:
    from bs4 import BeautifulSoup  # BeautifulSoup는 기본적으로 제공되지 않을 수 있습니다.
    # 의존성 없이 간단한 치환(정규식)으로 대체
    import re
    # <head> 에 CSS를 <style>로 삽입
    def inject_style(html, css_text):
        # link 태그를 style 태그로 교체
        html2 = re.sub(
            r'<link[^>]*href\s*=\s*["\']\./toss_main_CSS\.CSS["\'][^>]*>',
            '<style>' + css_text.replace('</', '<\/') + '</style>',
            html,
            flags=re.IGNORECASE
        )
        return html2
    with open(CSS_PATH, "r", encoding="utf-8") as f:
        css_text = f.read()
    html_inlined = inject_style(html_str, css_text)
    return HTML(html_inlined)

## STEP 1 — 베이스(상태바/헤더/메인 계좌 카드)

- 디자인 토큰(색/그림자/라운드)과 베이스 타이포그래피 세팅
- 상단 **Status Bar + Header** 구성
- 메인 **계좌 카드**(제목/잔액/버튼, 보조 행) 추가


In [ ]:
step1_body = """"<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style UI — Step</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
</head>
<body>
  <main class="app">
<!-- STATUS BAR -->
<div class="statusbar">
  <div>10:18</div>
  <div class="icons">📡 📶 <span class="pill">73%</span></div>
</div>

<!-- HEADER -->
<div class="header-row">
  <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
  <div class="right-btn">
    <span class="pill">결제</span> 🔔
  </div>
</div>

<!-- MAIN ACCOUNT CARD -->
<section class="card">
  <div class="account-main">
    <div>
      <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
      <div class="balance">1,222,222원</div>
    </div>
    <button class="btn">송금</button>
  </div>
  <div class="row">
    <div class="k">내 토스뱅크</div>
    <div class="v">쌓인 이자 33원 ▸</div>
  </div>
</section>
  </main>
</body>
</html>"

In [ ]:
# 파일 저장 & 미리보기
p = f"{BASE}/index_step1.html"
with open(p, "w", encoding="utf-8") as f:
    f.write(step1_body)
print("✔ 저장:", p)
display(inline_preview(step1_body))

## STEP 2 — 자주 쓰는 계좌 리스트 + 탭

- **칩(chip)**, **리스트 아이템**, **탭** 컴포넌트
- 금액/문구는 `clamp()` + 컨테이너 쿼리(cqi)로 크기 자동 조절


In [ ]:
step2_body = """"<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style UI — Step</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
</head>
<body>
  <main class="app">
<!-- STATUS BAR -->
<div class="statusbar">
  <div>10:18</div>
  <div class="icons">📡 📶 <span class="pill">73%</span></div>
</div>

<!-- HEADER -->
<div class="header-row">
  <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
  <div class="right-btn">
    <span class="pill">결제</span> 🔔
  </div>
</div>

<!-- MAIN ACCOUNT CARD -->
<section class="card">
  <div class="account-main">
    <div>
      <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
      <div class="balance">1,222,222원</div>
    </div>
    <button class="btn">송금</button>
  </div>
  <div class="row">
    <div class="k">내 토스뱅크</div>
    <div class="v">쌓인 이자 33원 ▸</div>
  </div>
</section><!-- FREQUENT ACCOUNTS -->
<div class="card" style="padding:18px 14px">
  <div class="section-title">자주 쓰는 계좌 <span style="float:right;color:#8a96a8">⚙️</span></div>
  <div class="list">
    <div class="item">
      <div class="name"><span class="chip">🐟</span> 1,200,000원<br><span class="muted" style="font-weight:700">굴비적금</span></div>
      <div></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip pink">🍡</span> 100,151,400원<br><span class="muted" style="font-weight:700">투자 모아보기 · 3개</span></div>
      <div class="right"><span class="pct">+4.0%</span></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip gold">🏆</span> 10,000원<br><span class="muted" style="font-weight:700">토스뱅크 도전통장</span></div>
      <div></div>
    </div>
  </div>
  <div class="row-tabs">
    <div class="tab">대출 1</div>
    <div class="tab">증권 5</div>
    <div class="tab">저축 6</div>
    <div class="tab">모든 계좌</div>
  </div>
</div>
  </main>
</body>
</html>"

In [ ]:
p = f"{BASE}/index_step2.html"
with open(p, "w", encoding="utf-8") as f:
    f.write(step2_body)
print("✔ 저장:", p)
display(inline_preview(step2_body))

## STEP 3 — 지출/카드값 스몰 카드

- 좌/우 정렬 레이아웃, 강조 숫자(`.big`), 배지 버튼


In [ ]:
step3_body = """"<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style UI — Step</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
</head>
<body>
  <main class="app">
<!-- STATUS BAR -->
<div class="statusbar">
  <div>10:18</div>
  <div class="icons">📡 📶 <span class="pill">73%</span></div>
</div>

<!-- HEADER -->
<div class="header-row">
  <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
  <div class="right-btn">
    <span class="pill">결제</span> 🔔
  </div>
</div>

<!-- MAIN ACCOUNT CARD -->
<section class="card">
  <div class="account-main">
    <div>
      <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
      <div class="balance">1,222,222원</div>
    </div>
    <button class="btn">송금</button>
  </div>
  <div class="row">
    <div class="k">내 토스뱅크</div>
    <div class="v">쌓인 이자 33원 ▸</div>
  </div>
</section><!-- FREQUENT ACCOUNTS -->
<div class="card" style="padding:18px 14px">
  <div class="section-title">자주 쓰는 계좌 <span style="float:right;color:#8a96a8">⚙️</span></div>
  <div class="list">
    <div class="item">
      <div class="name"><span class="chip">🐟</span> 1,200,000원<br><span class="muted" style="font-weight:700">굴비적금</span></div>
      <div></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip pink">🍡</span> 100,151,400원<br><span class="muted" style="font-weight:700">투자 모아보기 · 3개</span></div>
      <div class="right"><span class="pct">+4.0%</span></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip gold">🏆</span> 10,000원<br><span class="muted" style="font-weight:700">토스뱅크 도전통장</span></div>
      <div></div>
    </div>
  </div>
  <div class="row-tabs">
    <div class="tab">대출 1</div>
    <div class="tab">증권 5</div>
    <div class="tab">저축 6</div>
    <div class="tab">모든 계좌</div>
  </div>
</div><!-- SPENDING CARD -->
<div class="sp-16"></div>
<div class="small-card">
  <div>
    <div class="big">1,418,387원</div>
    <div class="title">9월에 쓴 돈</div>
  </div>
  <div class="badge">내역</div>
</div>

<div class="sp-8"></div>
<div class="small-card">
  <div>
    <div class="big">3,439,545원</div>
    <div class="title">9월 25일 낼 카드값</div>
  </div>
</div>
  </main>
</body>
</html>"

In [ ]:
p = f"{BASE}/index_step3.html"
with open(p, "w", encoding="utf-8") as f:
    f.write(step3_body)
print("✔ 저장:", p)
display(inline_preview(step3_body))

## STEP 4 — 신용점수 카드 + 3분할 빠른 액션

- 버튼 그룹을 **그리드(3열)**로 배치


In [ ]:
step4_body = """"<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style UI — Step</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
</head>
<body>
  <main class="app">
<!-- STATUS BAR -->
<div class="statusbar">
  <div>10:18</div>
  <div class="icons">📡 📶 <span class="pill">73%</span></div>
</div>

<!-- HEADER -->
<div class="header-row">
  <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
  <div class="right-btn">
    <span class="pill">결제</span> 🔔
  </div>
</div>

<!-- MAIN ACCOUNT CARD -->
<section class="card">
  <div class="account-main">
    <div>
      <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
      <div class="balance">1,222,222원</div>
    </div>
    <button class="btn">송금</button>
  </div>
  <div class="row">
    <div class="k">내 토스뱅크</div>
    <div class="v">쌓인 이자 33원 ▸</div>
  </div>
</section><!-- FREQUENT ACCOUNTS -->
<div class="card" style="padding:18px 14px">
  <div class="section-title">자주 쓰는 계좌 <span style="float:right;color:#8a96a8">⚙️</span></div>
  <div class="list">
    <div class="item">
      <div class="name"><span class="chip">🐟</span> 1,200,000원<br><span class="muted" style="font-weight:700">굴비적금</span></div>
      <div></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip pink">🍡</span> 100,151,400원<br><span class="muted" style="font-weight:700">투자 모아보기 · 3개</span></div>
      <div class="right"><span class="pct">+4.0%</span></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip gold">🏆</span> 10,000원<br><span class="muted" style="font-weight:700">토스뱅크 도전통장</span></div>
      <div></div>
    </div>
  </div>
  <div class="row-tabs">
    <div class="tab">대출 1</div>
    <div class="tab">증권 5</div>
    <div class="tab">저축 6</div>
    <div class="tab">모든 계좌</div>
  </div>
</div><!-- SPENDING CARD -->
<div class="sp-16"></div>
<div class="small-card">
  <div>
    <div class="big">1,418,387원</div>
    <div class="title">9월에 쓴 돈</div>
  </div>
  <div class="badge">내역</div>
</div>

<div class="sp-8"></div>
<div class="small-card">
  <div>
    <div class="big">3,439,545원</div>
    <div class="title">9월 25일 낼 카드값</div>
  </div>
</div><div class="card">
  <div style="display:flex;justify-content:space-between;align-items:center">
    <div style="font-weight:900;display:flex;align-items:center;gap:8px">내 신용점수 <span class="pill" style="background:#fff2d7;color:#8a5a00">변동 보기</span></div>
  </div>
  <div class="btn-row">
    <div class="btn-lite">계좌개설</div>
    <div class="btn-lite">카드발급</div>
    <div class="btn-lite">대출받기</div>
  </div>
</div>
  </main>
</body>
</html>"

In [ ]:
p = f"{BASE}/index_step4.html"
with open(p, "w", encoding="utf-8") as f:
    f.write(step4_body)
print("✔ 저장:", p)
display(inline_preview(step4_body))

## STEP 5 — 혜택 배너 + 2열 타일

- 배너/타일은 **카드 스타일 재사용**


In [ ]:
step5_body = """"<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style UI — Step</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
</head>
<body>
  <main class="app">
<!-- STATUS BAR -->
<div class="statusbar">
  <div>10:18</div>
  <div class="icons">📡 📶 <span class="pill">73%</span></div>
</div>

<!-- HEADER -->
<div class="header-row">
  <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
  <div class="right-btn">
    <span class="pill">결제</span> 🔔
  </div>
</div>

<!-- MAIN ACCOUNT CARD -->
<section class="card">
  <div class="account-main">
    <div>
      <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
      <div class="balance">1,222,222원</div>
    </div>
    <button class="btn">송금</button>
  </div>
  <div class="row">
    <div class="k">내 토스뱅크</div>
    <div class="v">쌓인 이자 33원 ▸</div>
  </div>
</section><!-- FREQUENT ACCOUNTS -->
<div class="card" style="padding:18px 14px">
  <div class="section-title">자주 쓰는 계좌 <span style="float:right;color:#8a96a8">⚙️</span></div>
  <div class="list">
    <div class="item">
      <div class="name"><span class="chip">🐟</span> 1,200,000원<br><span class="muted" style="font-weight:700">굴비적금</span></div>
      <div></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip pink">🍡</span> 100,151,400원<br><span class="muted" style="font-weight:700">투자 모아보기 · 3개</span></div>
      <div class="right"><span class="pct">+4.0%</span></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip gold">🏆</span> 10,000원<br><span class="muted" style="font-weight:700">토스뱅크 도전통장</span></div>
      <div></div>
    </div>
  </div>
  <div class="row-tabs">
    <div class="tab">대출 1</div>
    <div class="tab">증권 5</div>
    <div class="tab">저축 6</div>
    <div class="tab">모든 계좌</div>
  </div>
</div><!-- SPENDING CARD -->
<div class="sp-16"></div>
<div class="small-card">
  <div>
    <div class="big">1,418,387원</div>
    <div class="title">9월에 쓴 돈</div>
  </div>
  <div class="badge">내역</div>
</div>

<div class="sp-8"></div>
<div class="small-card">
  <div>
    <div class="big">3,439,545원</div>
    <div class="title">9월 25일 낼 카드값</div>
  </div>
</div><div class="card">
  <div style="display:flex;justify-content:space-between;align-items:center">
    <div style="font-weight:900;display:flex;align-items:center;gap:8px">내 신용점수 <span class="pill" style="background:#fff2d7;color:#8a5a00">변동 보기</span></div>
  </div>
  <div class="btn-row">
    <div class="btn-lite">계좌개설</div>
    <div class="btn-lite">카드발급</div>
    <div class="btn-lite">대출받기</div>
  </div>
</div><div class="sp-16"></div>
<div class="banner">
  <div class="mark">C</div>
  <div>
    <div class="headline">대신증권 신규고객 평생혜택</div>
    <div class="muted">국내주식 유관비용만 부과</div>
  </div>
</div>

<div class="row-head">OO님을 위해 준비했어요</div>
<div class="grid-two">
  <div class="tile">혜택 받는 신용카드</div>
  <div class="tile">혜택 받는 체크카드</div>
</div>
  </main>
</body>
</html>"

In [ ]:
p = f"{BASE}/index_step5.html"
with open(p, "w", encoding="utf-8") as f:
    f.write(step5_body)
print("✔ 저장:", p)
display(inline_preview(step5_body))

## STEP 6 — 심플 리스트 + 하단 네비게이션

- 리스트형 카드와 **sticky** 하단 내비게이션 바


In [ ]:
step6_body = """"<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style UI — Step</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
</head>
<body>
  <main class="app">
<!-- STATUS BAR -->
<div class="statusbar">
  <div>10:18</div>
  <div class="icons">📡 📶 <span class="pill">73%</span></div>
</div>

<!-- HEADER -->
<div class="header-row">
  <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
  <div class="right-btn">
    <span class="pill">결제</span> 🔔
  </div>
</div>

<!-- MAIN ACCOUNT CARD -->
<section class="card">
  <div class="account-main">
    <div>
      <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
      <div class="balance">1,222,222원</div>
    </div>
    <button class="btn">송금</button>
  </div>
  <div class="row">
    <div class="k">내 토스뱅크</div>
    <div class="v">쌓인 이자 33원 ▸</div>
  </div>
</section><!-- FREQUENT ACCOUNTS -->
<div class="card" style="padding:18px 14px">
  <div class="section-title">자주 쓰는 계좌 <span style="float:right;color:#8a96a8">⚙️</span></div>
  <div class="list">
    <div class="item">
      <div class="name"><span class="chip">🐟</span> 1,200,000원<br><span class="muted" style="font-weight:700">굴비적금</span></div>
      <div></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip pink">🍡</span> 100,151,400원<br><span class="muted" style="font-weight:700">투자 모아보기 · 3개</span></div>
      <div class="right"><span class="pct">+4.0%</span></div>
    </div>
    <div class="item">
      <div class="name"><span class="chip gold">🏆</span> 10,000원<br><span class="muted" style="font-weight:700">토스뱅크 도전통장</span></div>
      <div></div>
    </div>
  </div>
  <div class="row-tabs">
    <div class="tab">대출 1</div>
    <div class="tab">증권 5</div>
    <div class="tab">저축 6</div>
    <div class="tab">모든 계좌</div>
  </div>
</div><!-- SPENDING CARD -->
<div class="sp-16"></div>
<div class="small-card">
  <div>
    <div class="big">1,418,387원</div>
    <div class="title">9월에 쓴 돈</div>
  </div>
  <div class="badge">내역</div>
</div>

<div class="sp-8"></div>
<div class="small-card">
  <div>
    <div class="big">3,439,545원</div>
    <div class="title">9월 25일 낼 카드값</div>
  </div>
</div><div class="card">
  <div style="display:flex;justify-content:space-between;align-items:center">
    <div style="font-weight:900;display:flex;align-items:center;gap:8px">내 신용점수 <span class="pill" style="background:#fff2d7;color:#8a5a00">변동 보기</span></div>
  </div>
  <div class="btn-row">
    <div class="btn-lite">계좌개설</div>
    <div class="btn-lite">카드발급</div>
    <div class="btn-lite">대출받기</div>
  </div>
</div><div class="sp-16"></div>
<div class="banner">
  <div class="mark">C</div>
  <div>
    <div class="headline">대신증권 신규고객 평생혜택</div>
    <div class="muted">국내주식 유관비용만 부과</div>
  </div>
</div>

<div class="row-head">OO님을 위해 준비했어요</div>
<div class="grid-two">
  <div class="tile">혜택 받는 신용카드</div>
  <div class="tile">혜택 받는 체크카드</div>
</div><div class="sp-16"></div>
<div class="card list-simple">
  <div class="li"><span class="icon-square"></span> 할인·적립 쿠폰</div>
  <div class="li"><span class="icon-square"></span> 애플허브</div>
  <div class="li"><span class="icon-square"></span> 토스신용카드</div>
</div>

<nav class="navbar">
  <div class="navitem"><span class="dot"></span>홈</div>
  <div class="navitem active"><span class="dot"></span>혜택</div>
  <div class="navitem"><span class="dot"></span>쇼핑</div>
  <div class="navitem"><span class="dot"></span>증권</div>
  <div class="navitem"><span class="dot"></span>전체</div>
</nav>
  </main>
</body>
</html>"

In [ ]:
p = f"{BASE}/index_step6.html"
with open(p, "w", encoding="utf-8") as f:
    f.write(step6_body)
print("✔ 저장:", p)
display(inline_preview(step6_body))

## BONUS — Fallback → Enhanced 전략 복습

- **Fallback**: 컨테이너 쿼리를 지원하지 않는 브라우저용으로 `vw` 기반 `clamp()` 제공  
- **Enhanced**: `@supports (container-type: inline-size)`에서 `cqi` 단위로 정밀 제어  
- 모바일 초소형 화면은 별도 `@media (max-width:360px)`로 하한선 보정


## 최종 조립 — 제공된 원문 HTML/CSS 전체 확인

아래 셀은 **원문 그대로의 전체 HTML**을 `index.html`로 저장하고 노트북 안에서 동일하게 미리봅니다.


In [ ]:
html_final = r""""<!doctype html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Toss-style Mobile UI (Static HTML/CSS)</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
  <link href="./toss_main_CSS.CSS" rel="stylesheet">
  
</head>
<body>
  <main class="app">
    <!-- STATUS BAR -->
    <div class="statusbar">
      <div>10:18</div>
      <div class="icons">📡 📶 <span class="pill">73%</span></div>
    </div>

    <!-- HEADER -->
    <div class="header-row">
      <div style="display:flex;align-items:center;gap:8px;font-weight:800;color:#3a4a62">📁 할 일</div>
      <div class="right-btn">
        <span class="pill">결제</span>
        🔔
      </div>
    </div>

    <!-- MAIN ACCOUNT CARD -->
    <section class="card">
      <div class="account-main">
        <div>
          <div class="account-title"><span class="logo"></span> 토스뱅크 통장 · 주계좌</div>
          <div class="balance">1,222,222원</div>
        </div>
        <button class="btn">송금</button>
      </div>
      <div class="row">
        <div class="k">내 토스뱅크</div>
        <div class="v">쌓인 이자 33원 ▸</div>
      </div>
    </section>

    <!-- FREQUENT ACCOUNTS -->
    <div class="card" style="padding:18px 14px">
      <div class="section-title">자주 쓰는 계좌 <span style="float:right;color:#8a96a8">⚙️</span></div>
      <div class="list">
        <div class="item">
          <div class="name"><span class="chip">🐟</span> 1,200,000원<br><span class="muted" style="font-weight:700">굴비적금</span></div>
          <div></div>
        </div>
        <div class="item">
          <div class="name"><span class="chip pink">🍡</span> 100,151,400원<br><span class="muted" style="font-weight:700">투자 모아보기 · 3개</span></div>
          <div class="right"><span class="pct">+4.0%</span></div>
        </div>
        <div class="item">
          <div class="name"><span class="chip gold">🏆</span> 10,000원<br><span class="muted" style="font-weight:700">토스뱅크 도전통장</span></div>
          <div></div>
        </div>
      </div>

      <div class="row-tabs">
        <div class="tab">대출 1</div>
        <div class="tab">증권 5</div>
        <div class="tab">저축 6</div>
        <div class="tab">모든 계좌</div>
      </div>
    </div>

    <!-- SPENDING CARD -->
    <div class="sp-16"></div>
    <div class="small-card">
      <div>
        <div class="big">1,418,387원</div>
        <div class="title">9월에 쓴 돈</div>
      </div>
      <div class="badge">내역</div>
    </div>

    <div class="sp-8"></div>
    <div class="small-card">
      <div>
        <div class="big">3,439,545원</div>
        <div class="title">9월 25일 낼 카드값</div>
      </div>
    </div>

      <div class="card">
        <div style="display:flex;justify-content:space-between;align-items:center">
          <div style="font-weight:900;display:flex;align-items:center;gap:8px">내 신용점수 <span class="pill" style="background:#fff2d7;color:#8a5a00">변동 보기</span></div>
        </div>
        <div class="btn-row">
          <div class="btn-lite">계좌개설</div>
          <div class="btn-lite">카드발급</div>
          <div class="btn-lite">대출받기</div>
        </div>
      </div>

      <div class="sp-16"></div>
      <div class="banner">
        <div class="mark">C</div>
        <div>
          <div class="headline">대신증권 신규고객 평생혜택</div>
          <div class="muted">국내주식 유관비용만 부과</div>
        </div>
      </div>

      <div class="row-head">OO님을 위해 준비했어요</div>
      <div class="grid-two">
        <div class="tile">혜택 받는 신용카드</div>
        <div class="tile">혜택 받는 체크카드</div>
      </div>

      <div class="sp-16"></div>
      <div class="card list-simple">
        <div class="li"><span class="icon-square"></span> 할인·적립 쿠폰</div>
        <div class="li"><span class="icon-square"></span> 애플허브</div>
        <div class="li"><span class="icon-square"></span> 토스신용카드</div>
      </div>

      <nav class="navbar">
        <div class="navitem"><span class="dot"></span>홈</div>
        <div class="navitem active"><span class="dot"></span>혜택</div>
        <div class="navitem"><span class="dot"></span>쇼핑</div>
        <div class="navitem"><span class="dot"></span>증권</div>
        <div class="navitem"><span class="dot"></span>전체</div>
      </nav>
    </section>
  </main>
</body>
</html>
"""
# 저장
with open(f"{BASE}/index.html", "w", encoding="utf-8") as f:
    f.write(html_final)
print("✔ 저장:", f"{BASE}/index.html")
# 미리보기
display(inline_preview(html_final))

### 배포/미니 과제
- VS Code의 **Live Server** 확장으로 `index.html`을 모바일 뷰에서 확인해 보세요.
- 각 섹션의 텍스트/이모지/아이콘을 바꿔 **본인 용도**로 커스터마이즈해 보세요.
- **접근성**(명도 대비, 포커스 스타일) 개선 포인트를 찾아 추가해 보세요.
